In [2]:
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import SymLogNorm
import h5py
from tqdm import tqdm

In [66]:
basePath = 'sims.TNG/TNG100-1/output'

snap = 99
halo_ID = 172650

tree  = il.sublink.loadTree(basePath,snap,halo_ID)
stars = il.snapshot.loadSubhalo(basePath,snap,halo_ID,'stars',['Coordinates','ParticleIDs','Masses'])

In [162]:
#### First, find all subhalos that come close to Main Progenitor ###

tree_main = il.sublink.loadTree(basePath, snap, halo_ID, fields=['SnapNum','SubfindID'], onlyMPB=True)
Snap_to_Subfind = dict(zip(tree_main['SnapNum'][::-1], tree_main['SubfindID'][::-1]))

lookup_subhalos = []
lookup_snapnum  = []
for s in tqdm(Snap_to_Subfind.keys(), leave=True):

    all_subhalos = il.groupcat.loadSubhalos(basePath,s,fields=['SubhaloHalfmassRad','SubhaloPos', 'SubhaloMass'])

    main_subhalo_halfmassrad = all_subhalos['SubhaloHalfmassRad'][Snap_to_Subfind[s]]
    virial_radius = main_subhalo_halfmassrad * 10

    all_subhalo_radius  = np.sum((all_subhalos['SubhaloPos']-all_subhalos['SubhaloPos'][Snap_to_Subfind[s]])**2,axis=1)**0.5

    subhalos_keep = np.where(all_subhalo_radius <= virial_radius)[0]

    lookup_subhalos.extend(subhalos_keep)
    lookup_snapnum.extend([s]*len(subhalos_keep))

np.savetxt(f'lookup_subhalos_{halo_ID}.txt', lookup_subhalos)
np.savetxt(f'lookup_snapnum_ {halo_ID}.txt', lookup_snapnum)

  9%|▉         | 9/99 [02:40<32:33, 21.71s/it]

In [ ]:
#### First, find all subhalos that come close to Main Progenitor ###

tree_main = il.sublink.loadTree(basePath, snap, halo_ID, fields=['SnapNum','SubfindID'], onlyMPB=True)
Snap_to_Subfind = dict(zip(tree_main['SnapNum'][::-1], tree_main['SubfindID'][::-1]))

lookup_subhalos = []
lookup_snapnum  = []
for s in tqdm(Snap_to_Subfind.keys(), leave=True):

    all_subhalos = il.groupcat.loadSubhalos(basePath,s,fields=['SubhaloHalfmassRad','SubhaloPos', 'SubhaloMass'])

    main_subhalo_halfmassrad = all_subhalos['SubhaloHalfmassRad'][Snap_to_Subfind[s]]
    virial_radius = main_subhalo_halfmassrad * 5

    all_subhalo_radius  = np.sum((all_subhalos['SubhaloPos']-all_subhalos['SubhaloPos'][Snap_to_Subfind[s]])**2,axis=1)**0.5

    subhalos_keep = np.where(all_subhalo_radius <= virial_radius)[0]

    lookup_subhalos.extend(subhalos_keep)
    lookup_snapnum.extend([s]*len(subhalos_keep))

np.savetxt(f'lookup_subhalos_{halo_ID}_rad5.txt', lookup_subhalos)
np.savetxt(f'lookup_snapnum_ {halo_ID}_rad5.txt', lookup_snapnum)

In [85]:
lookup_subhalos = np.loadtxt(f'lookup_subhalos_{halo_ID}.txt')
lookup_snapnum  = np.loadtxt(f'lookup_snapnum_ {halo_ID}.txt')

In [110]:
ID_init        = tree['SubhaloID'][0]
DescendantID   = tree['DescendantID'] - ID_init
tree_SubfinID  = tree['SubfindID']
tree_SubHaloID = tree['SubhaloID'] - ID_init

Subhalo_to_Subfind = dict(zip(tree_SubHaloID, tree_SubfinID))

subfind_descendant = tree_SubfinID[np.where(DescendantID <= 100)[0]]

In [118]:
np.isin(subfind_descendant, lookup_subhalos)

array([ True,  True,  True, ...,  True, False,  True])

In [119]:
subfind_descendant[-2]

175595

In [120]:
np.where(tree_SubfinID == 175595)

(array([50512]),)

In [130]:
(tree['SubhaloPos'][50512] **2).sum()**0.5-(tree['SubhaloPos'][1] **2).sum()**0.5

-53.99897072513704

In [128]:
(tree['SubhaloPos'][0] **2).sum()**0.5

72520.19018176939

In [109]:
tree_SubfinID

array([ 172650,  173164,  174699, ...,  175419, 2780422,  173229],
      dtype=int32)

In [92]:
DescendantID

array([-40000000600096289,                  0,                  1, ...,
                    50513,              50514,                  0])

In [95]:
np.isin(idx, lookup_subhalos)

array([False, False, False, ..., False, False, False])

In [86]:
lookup_subhalos

array([  9297.,   4811.,   6577., ..., 175210., 175211., 175216.])

In [83]:
!ls

 examples	   'lookup_snapnum_ 172650.txt'      README.txt
 illustris_idl	    lookup_subhalos_172650.txt	     sims.illustris
 illustris_julia    moreColors.png		     sims.other
 illustris_matlab   progenitorID_subhalo172650.txt   sims.TNG
 illustris_python   progenitorID_subhalo239993.txt   SubhaloID_stars.txt


In [57]:
subhalos_keep[0].shape

(180,)

In [36]:
subhalo_halfmassrad

9.209394

In [15]:
subhalos = il.groupcat.loadSubhalos(basePath,s,fields=['SubhaloFlag', 'SubhaloMass'])

In [17]:
subhalos['SubhaloMass'].max()

0.62153184

In [ ]:
progenitorID = np.zeros(stars['ParticleIDs'].shape) - 1
all_stars = stars['ParticleIDs']
# Iterate over all snapshots
for s in range(100):
    print(s)

    subhalos = il.groupcat.loadSubhalos(basePath,s,fields=['SubhaloFlag', 'SubhaloMass'])
    subfindID = np.where( (subhalos['SubhaloFlag'] == 1) & (subhalos['SubhaloMass'] >= 1) )[0]

    # Iterate over all subhalos
    for i in tqdm(subfindID, leave=True): 

        starsID_here = il.snapshot.loadSubhalo(basePath,s,i,'stars',['ParticleIDs'])
        
        not_already_IDed = np.where(progenitorID == -1)[0]
        stars_in = np.isin(all_stars[not_already_IDed], starsID_here)
        
        indices = not_already_IDed[stars_in]

        progenitorID[indices] = i


0


0it [00:00, ?it/s]


1


0it [00:00, ?it/s]


2


...

45


  4%|▍         | 3997/101908 [02:22<1:00:07, 27.14it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 67%|██████▋   | 67807/101908 [25:07<06:20, 89.70it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 39%|███▉      | 39857/102165 [17:14<21:02, 49.35it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Note

48


  1%|          | 1211/102320 [00:43<44:42, 37.70it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 63%|██████▎   | 64645/102320 [22:28<13:05, 47.94it/s] 


KeyboardInterrupt: 

In [ ]:
np.savetxt('text_progenitorID.txt', progenitorID, fmt='%d')